<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

卷積網路

執行階段 -> 變更執行階段類型 -> GPU 運算速度快10倍

卷積層[https://keras.io/zh/layers/convolutional/ ] \
池化層[https://keras.io/zh/layers/pooling/ ]

In [ ]:
from tensorflow.keras.datasets.cifar10 import load_data
(x_train, y_train), (x_test, y_test) = load_data()

若要把minst資料集帶進來，要從二維改成三維

mnist: 60000, 28, 28 -> 60000, 28, 28, 1

In [ ]:
# 彩色圖像 長, 寬, 通道數(RGB)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
print(len(y_train))
print(y_train)

In [ ]:
trans = [
         "airplane",										
         "automobile",										
         "bird",										
         "cat",										
         "deer",										
         "dog",										
         "frog",										
         "horse",										
         "ship",										
         "truck"
]

In [ ]:
import random
import matplotlib.pyplot as plt
idx = random.randint(0, 49999)
print(y_train[idx])
print(trans[y_train[idx][0]])
plt.imshow(x_train[idx])

Flatten 攤平成同一維度

kernal_size:3 (3*3的filter) \
strides 步伐=1(預設值) \
padding: same 給邊距以維持原圖大小32x32, \
如果設定valid則代表不留padding,導致filter吃不到最邊緣，會讓原圖變成30x30 \

$$
3*3*3*64 = 1728, 一個過濾器會附一個bias, 64個門檻要移回來, activation=64, 1728+64=1792
$$

          # flatten()攤平後, 長*寬*高= 欄位數(輸出空間維度)
          # 4096*128+128= 524416


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
layers = [
          # 一個filter(27)*64 + 64(bias) = 1792
          Conv2D(64, 3, padding="same", activation="relu", input_shape=(32, 32, 3)),
          MaxPooling2D(),
          # 3*3*64 * 128 + 128 = 73856
          Conv2D(128, 3, padding="same", activation="relu"),
          MaxPooling2D(),
          # 3*3*128 * 256 + 256 = 295168
          Conv2D(256, 3, padding="same", activation="relu"),
          MaxPooling2D(),
          # 3*3*256 * 512 + 512 = 1180160
          Conv2D(512, 3, padding="same", activation="relu"),
          MaxPooling2D(),
          Flatten(),
          Dense(128, activation="relu"),
          Dropout(0.25),
          Dense(10, activation="softmax")

]
model = Sequential(layers)
model.summary()

In [ ]:
# one-hot encoding讓它自己搞定, 不需要自己做One-Hot
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])


In [ ]:
x_train_norm = x_train / 255
x_test_norm = x_test / 255

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
             EarlyStopping(patience=5, restore_best_weights=True),
             ModelCheckpoint("cnn.h5", save_best_only=True)
]
model.fit(x_train_norm,
          y_train,
          batch_size=200,
          epochs=50,
          validation_split=0.1,
          callbacks=callbacks,
          verbose=2)

In [ ]:
model.evaluate(x_test_norm, y_test)

In [ ]:
pre = model.predict_classes(x_test_norm)
pre

ndarray.reshape() 更改陣列的尺寸\
設定-1, 會自動計算array內的數量 \
在這裡是把y_test從一維ndarray的10000筆list改成一個list有10000筆數字

In [ ]:
print(y_test.shape)
print(y_test)
print(pre.shape)
y_test_r = y_test.reshape(-1)
print(y_test_r)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

mat = confusion_matrix(y_test_r, pre)
pd.DataFrame(mat,
             columns=[ trans[i] + "(預測)" for i in range(10)],
             index=[ trans[i] + "(正確)" for i in range(10)])

In [ ]:
import numpy as np
idx = np.nonzero(pre != y_test_r)[0][300:500]
print(np.nonzero(pre != y_test_r)[0].size)
false_img = x_test[idx]
false_pre = pre[idx]
false_label = y_test_r[idx]
plt.figure(figsize=(20, 60))
width = 10
height = len(idx) // width + 1
for i in range(len(idx)):
    plt.subplot(height, width, i+1)
    plt.imshow(false_img[i])
    t = "[Ori]:{}\n[Pre]:{}".format(trans[false_label[i]], trans[false_pre[i]])
    plt.title(t)
    plt.axis("off")

In [ ]:
from PIL import Image
import requests
headers ={
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"
}
url = input("url: ")
response = requests.get(url, headers=headers, stream=True, verify=False)
img = Image.open(response.raw).resize((32, 32)).convert("L").convert("RGB")
img_np = np.array(img)
img_np_r = img_np.reshape(1, 32, 32, 3) / 255.0
proba = model.predict(img_np_r)[0]
for i in range(10):
    print(trans[i], "的機率:", proba[i])
ans = model.predict_classes(img_np_r)[0]
print("答案:", trans[ans])
plt.imshow(img_np)